<a href="https://colab.research.google.com/github/lucasabbruzzini/Portfolio/blob/main/DL_GPW_2_Step_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import seaborn as sns
import matplotlib.pyplot as plt
import random
try:
    import keras_tuner as kt
except:
    !pip install keras_tuner
    import keras_tuner as kt


# Set seeds for reproducibility
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
tf.config.experimental.enable_op_determinism()

sns.set(style="whitegrid")

##3.a) Preprocess data

Building upon the foundational development in Step 2, we have engineered two classes: Asset and Portfolio. These classes represent a sophisticated object-oriented approach to managing financial data, reflecting a deliberate design choice rooted in software engineering principles and financial modeling requirements. The decision to implement classes in this context stems from their capacity to encapsulate data and behavior, providing a robust framework for handling complex financial datasets. This approach offers several advanced benefits:

Data Encapsulation: The Asset class consolidates all relevant financial metrics (e.g., prices, returns, volatility) into a single object, eliminating the need for disparate variable management across multiple assets. This enhances data integrity and reduces cognitive overhead.
Code Reusability: The class design facilitates the instantiation of multiple Asset objects (e.g., for SPY, TLT) with uniform behavior, leveraging polymorphism and inheritance potential to avoid redundant code implementation.
Extensibility: The object-oriented paradigm allows for seamless integration of advanced financial metrics—such as dividend yields or risk-adjusted performance measures like the Sharpe ratio—through method augmentation, ensuring scalability without architectural overhaul.
Asset Aggregation: The Portfolio class centralizes the management of diverse Asset objects, functioning as a structured container that simplifies portfolio-level operations and enhances data coherence.
Analytical Facilitation: By consolidating returns into a single DataFrame, it supports advanced econometric analyses, such as correlation studies or input preparation for machine learning models (e.g., LSTM networks for return prediction), optimizing computational efficiency.
Flexibility and Scalability: The class structure permits the incorporation of portfolio and risk management techniques to be explored in the next courses.
The Asset class serves as a modular entity, encapsulating the attributes and behaviors of individual financial instruments, such as equities or exchange-traded funds (e.g., SPY, a proxy for the S&P 500). This class is architected to manage intricate financial computations, including the retrieval of historical price series, computation of daily logarithmic returns, and estimation of volatility metrics. It initializes with parameters such as ticker symbol, temporal range, data frequency (defaulting to daily), and a rolling window parameter for statistical analysis, subsequently populating instance variables with processed data.

The Portfolio class acts as a higher-level aggregator, synthesizing multiple Asset instances into a cohesive investment portfolio. It is initialized with a portfolio identifier, ownership details, and a list of ticker symbols, subsequently populating its asset collection. This class provides analytical capabilities, including the aggregation of asset returns into a unified DataFrame and the generation of a comprehensive portfolio valuation report.

In [ ]:
class Asset:
    def __init__(self, ticker, start_date, end_date, frequency='1d', window=20):
        # Here set up the basics for this asset:
        self.ticker = ticker
        self.start_date = pd.Timestamp(start_date)
        self.end_date = pd.Timestamp(end_date)
        self.frequency = frequency
        self.window = window
        # Now we'll grab the price data, compute returns, and figure out the volatility.
        self.prices = self._download_data()
        self.returns = self.calculate_returns()
        self.volatility = self.calculate_volatility()

    def _download_data(self):
        import time
        retries = 3  # We'll give it 3 tries in case something goes wrong with the download. Note that there was difficults during the creation of this code due to too many requests
        for attempt in range(retries):
            try:
                print(f"Trying to grab price data for {self.ticker} from {self.start_date} to {self.end_date}...")
                time.sleep(2)  # A little pause to avoid overwhelming Yahoo Finance's servers and get 429 response
                data = yf.download(self.ticker, start=self.start_date, end=self.end_date, interval=self.frequency)
                if data.empty:
                    print(f"Uh-oh, no data came back for {self.ticker}.")
                    raise ValueError(f"No price data for {self.ticker}. Check ticker or connection.")
                print(f"Got the data for {self.ticker} successfully!")
                # We'll use the adjusted close price if available; otherwise, the regular close.
                return data['Adj Close'].squeeze() if 'Adj Close' in data else data['Close'].squeeze()
            except Exception as e:
                if attempt < retries - 1:
                    print(f"Download failed for {self.ticker} (attempt {attempt + 1}/{retries}): {e}. Let's try again...")
                    time.sleep(2 ** attempt)  # Wait a bit longer each time before retrying.
                else:
                    print(f"That's it, we couldn't get the data for {self.ticker}: {e}")
                    raise ValueError(f"No price data for {self.ticker} after {retries} attempts. Check ticker or connection.")

    def calculate_returns(self):
        # This calculates the percentage change in prices day-to-day to get the returns, filling any missing values with 0.
        return self.prices.pct_change().fillna(0)

    def calculate_volatility(self, annualize=True):
        # Here we calculate the volatility, which tells us how much the returns fluctuate over a 20-day window (or whatever window we set).
        volatility = self.returns.rolling(window=self.window).std()
        # annualize it
        return volatility * np.sqrt(252) if annualize else volatility

class Portfolio:
    def __init__(self, name, owner, tickers, frequency='1d', window=20):
        # We're setting up a portfolio with a name, owner, and a list of tickers (like SPY, TLT, etc.).
        self.name = name
        self.owner = owner
        self.frequency = frequency  # How often we want the data (daily by default).
        self.window = window  # Window for calculations like volatility.
        self.assets = []  # This will hold our Asset objects.
        self.tickers = tickers
        # Let's create an Asset object for each ticker, covering the date range from 2008 to 2022.
        self.assets = [Asset(ticker, "2008-01-01", "2022-12-30", self.frequency, self.window) for ticker in self.tickers]

    def get_returns_df(self):
        # This method gathers the returns for all assets in the portfolio and puts them into a single DataFrame.
        if not self.assets:
            raise ValueError("Oops, there are no assets in the portfolio yet. Try creating some first!")
        return pd.DataFrame({asset.ticker: asset.returns for asset in self.assets}).fillna(0)

    def display_portfolio(self):
        # This prints out a summary of the portfolio, showing the total value and the latest price for each asset.
        if not self.assets:
            raise ValueError("Hold on, we don't have any assets to display. Create some assets first!")
        prices_df = pd.DataFrame({asset.ticker: asset.prices for asset in self.assets}).dropna(how='all')
        print(f"\nPortfolio: {self.name}")
        print(f"Owner: {self.owner}")
        print(f"Date Range: 2008-01-01 to 2022-12-30")
        print(f"Total Value (latest prices): ${prices_df.iloc[-1].sum():,.2f}")
        print("Assets:")
        for ticker in prices_df.columns:
            price = prices_df[ticker].iloc[-1] if not prices_df[ticker].isna().all() else 0
            print(f"  - {ticker}: ${price:,.2f}")

In [ ]:
# Lets choose the tickers and initialize our Portfolio
tickers = ["SPY", "TLT", "SHY", "GLD", "DBO"]
portfolio = Portfolio("Growth Fund", "John Doe", tickers)
portfolio.display_portfolio()
df_ret = portfolio.get_returns_df()
print("Portfolio creation completed.")

##**3.b) Build multi-output Model:Train and Test multi-output Model:

The MultiOutputLSTM class is a custom implementation for multi-output time series forecasting, designed to predict future returns for multiple stock tickers simultaneously using a Long Short-Term Memory (LSTM) neural network as studied in module 5. It leverages historical stock return data, constructs technical indicators (e.g., moving averages, volatility), and employs a deep learning architecture to capture temporal dependencies. The class integrates data preprocessing, model building, hyperparameter tuning, training, and performance evaluation, making it a comprehensive tool for financial time series analysis.

Key Components

Data Preparation:

Features: The gen_predictors method generates inputs like 10-day and 50-day moving averages and 20-day volatility, alongside a 25-day future return as the target (Ret25). These are standard financial indicators capturing trend and risk.
Splitting: Data is split chronologically—pre-2018 for training/validation (70%/30%) and 2018–2022 for testing—reflecting a realistic out-of-sample evaluation.
Scaling: Inputs are standardized (StandardScaler), and outputs are normalized per ticker (MinMaxScaler) to stabilize LSTM training. Lagging: The create_lags method creates sequences of window_size (default 30) timesteps, enabling the LSTM to learn from historical patterns.
Model Architecture:

The LSTM stack consists of three layers (two with return_sequences=True, one without), followed by dense layers (50, 20, and output nodes equal to the number of tickers). This depth captures complex temporal relationships, while dropout mitigates overfitting. Loss is mean squared error (MSE), optimized with Adam, suitable for regression tasks like return prediction.
Training and Evaluation:

Early stopping (patience=10) and learning rate scheduling (ReduceLROnPlateau) enhance training efficiency.

The analyze_performance method visualizes predictions versus actuals, aiding interpretability.

Keras Tuner Integration

The tune_hyperparameters method uses keras_tuner’s BayesianOptimization to optimize three hyperparameters:

units_lstm: Integer range [50, 300] (step 50). Controls LSTM layer capacity—larger values increase expressivity but risk overfitting.
n_dropout: Float range [0.0, 0.5] (step 0.1). Dropout rate balances regularization; higher values reduce overfitting but may underfit if excessive.
hp_lr: Float range [1e-5, 1e-2] (log sampling). Learning rate governs convergence speed—log sampling explores a wide range efficiently.
Parameter Choices and Non-Optimal Settings

Default Parameters: units_lstm=150, n_dropout=0.1, hp_lr=1e-3, epochs=20, and max_trials=5 (in tuning). These are reasonable starting points but not necessarily optimal:
units_lstm=150 balances capacity and complexity but may not suit all datasets.
n_dropout=0.1 is conservative; higher values might improve generalization.
hp_lr=1e-3 is a common default, but the optimal rate varies by problem.
Epochs and Max Trials:

epochs=20 limits training duration, potentially halting before full convergence, especially for noisy financial data requiring longer learning.
max_trials=5 restricts hyperparameter exploration, risking suboptimal configurations.
These conservative values reflect computational limitations (e.g., CPU/GPU availability, memory). Ideally, epochs could be 50–100, and max_trials 20–50, allowing deeper training and broader tuning. Modern hardware (e.g., TPUs) or cloud resources could support this, but without such access, we trade optimality for feasibility.

In [ ]:
class MultiOutputLSTM:
    def __init__(self, tickers, window_size=30, train_split=0.7, val_split=0.3, units_lstm=150, n_dropout=0.1, hp_lr=1e-3, epochs=20):
        # Here’s where we set up the basics: tickers are the stocks we’re tracking, window_size is how much past data we look at.
        self.tickers = tickers
        self.window_size = window_size
        self.train_split = train_split  # Splitting data: 70% for training by default.
        self.val_split = val_split      # 30% for validation.
        self.units_lstm = units_lstm    # Number of LSTM units—think of it as the brain size of our model.
        self.n_dropout = n_dropout      # Dropout rate to keep the model from overfitting.
        self.hp_lr = hp_lr              # Learning rate—how fast the model learns.
        self.scaler_input = StandardScaler()  # Scales our input data to keep things balanced.
        self.scalers_output = {ticker: MinMaxScaler() for ticker in tickers}  # One scaler per ticker for output.
        self.model = None               # Placeholder for our LSTM model—we’ll build it later.
        self.es = EarlyStopping(monitor="val_loss", mode="min", patience=10, restore_best_weights=True)  # Stops training if we’re stuck.
        self.lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6)  # Tweaks learning rate if needed.
        self.n_epochs = epochs          # How many times we’ll train the model. Keep in mind we had to keep this low due to tecnology and time constrains
        # These will hold our training, validation, and test data once we prepare it.
        self.X_train = None
        self.y_train = None
        self.train_time = None
        self.X_val = None
        self.y_val = None
        self.val_time = None
        self.X_test = None
        self.y_test = None
        self.test_time = None
        self.y_test_original_lagged = None  # Keeps the original test returns for comparison.

    def gen_predictors(self, df_ret):
        # This method creates features (predictors) from our stock returns data.
        df = df_ret.copy()  # Don’t mess with the original data!
        for ticker in self.tickers:
            # For each stock, we’re making some cool indicators:
            df[f"{ticker}_MA_10"] = df[ticker].rolling(10).mean()  # 10-day moving average.
            df[f"{ticker}_MA_50"] = df[ticker].rolling(50).mean()  # 50-day moving average.
            df[f"{ticker}_Volatility_20"] = df[ticker].rolling(20).std()  # 20-day volatility.
            df[f"{ticker}_Ret25"] = df[ticker].rolling(25).apply(lambda x: np.prod(1 + x) - 1).shift(-25)  # 25-day future return.
        df = df.dropna()  # Drop rows with missing values—clean slate!
        # Grab our predictors (X) and target (y) data.
        X = df[[f"{ticker}_MA_10" for ticker in self.tickers] +
               [f"{ticker}_MA_50" for ticker in self.tickers] +
               [f"{ticker}_Volatility_20" for ticker in self.tickers]].values
        y = df[[f"{ticker}_Ret25" for ticker in self.tickers]].values
        return X, y, df.index  # Return predictors, targets, and dates.

    def prepare_data(self, X, y, dates):
        # Time to split our data into training, validation, and test sets based on dates.
        pre_2018_mask = dates <= pd.Timestamp("2017-12-31")  # Everything before 2018 for training/validation.
        test_mask = (dates >= pd.Timestamp("2018-01-01")) & (dates <= pd.Timestamp("2022-12-30"))  # 2018-2022 for testing.
        pre_2018_X, pre_2018_y, pre_2018_dates = X[pre_2018_mask], y[pre_2018_mask], dates[pre_2018_mask]
        test_X, test_y, test_dates = X[test_mask], y[test_mask], dates[test_mask]

        # Split pre-2018 data into training and validation.
        train_size = int(len(pre_2018_y) * self.train_split)
        X_train_set = pre_2018_X[:train_size]
        X_val_set = pre_2018_X[train_size:]
        y_train_set = pre_2018_y[:train_size]
        y_val_set = pre_2018_y[train_size:]
        X_test_set, y_test_set = test_X, test_y
        self.val_time = pre_2018_dates[train_size:]  # Save validation dates.
        self.test_time = test_dates  # Save test dates.

        # Scale the data so everything’s on the same playing field.
        X_train_scaled = self.scaler_input.fit_transform(X_train_set)
        X_val_scaled = self.scaler_input.transform(X_val_set)
        X_test_scaled = self.scaler_input.transform(X_test_set)

        # Scale the outputs (returns) for each ticker separately.
        y_train_scaled = np.column_stack([self.scalers_output[ticker].fit_transform(y_train_set[:, i].reshape(-1, 1))
                                         for i, ticker in enumerate(self.tickers)])
        y_val_scaled = np.column_stack([self.scalers_output[ticker].transform(y_val_set[:, i].reshape(-1, 1))
                                       for i, ticker in enumerate(self.tickers)])
        y_test_scaled = np.column_stack([self.scalers_output[ticker].transform(y_test_set[:, i].reshape(-1, 1))
                                        for i, ticker in enumerate(self.tickers)])

        self.y_test_original_lagged = y_test_set  # Keep the unscaled test returns handy.
        return X_train_scaled, X_val_scaled, X_test_scaled, y_train_scaled, y_val_scaled, y_test_scaled

    def create_lags(self, X_scaled, y_scaled, dates):
        # This creates "lagged" sequences—think of it as sliding windows of past data.
        X_lagged, y_lagged, time_lagged = [], [], []
        for i in range(self.window_size, len(y_scaled)):
            X_lagged.append(X_scaled[i - self.window_size:i])  # Grab the past window_size days.
            y_lagged.append(y_scaled[i])  # The target for that window.
            time_lagged.append(dates[i])  # The corresponding date.
        return np.array(X_lagged), np.array(y_lagged), np.array(time_lagged)

    def build_model(self, hp, input_shape):
        # Here’s where we define our LSTM model architecture with some tunable hyperparameters.
        units_lstm = hp.Int('units_lstm', min_value=50, max_value=300, step=50)  # How many LSTM units?
        n_dropout = hp.Float('n_dropout', min_value=0.0, max_value=0.5, step=0.1)  # Dropout rate?
        hp_lr = hp.Float('hp_lr', min_value=1e-5, max_value=1e-2, sampling='log')  # Learning rate?

        model = Sequential([
            Input(shape=input_shape),  # Define how big our input is.
            LSTM(units_lstm, return_sequences=True),  # First LSTM layer—keeps sequences.
            Dropout(n_dropout),  # Prevent overfitting.
            LSTM(units_lstm, return_sequences=True),  # Second LSTM layer.
            Dropout(n_dropout),
            LSTM(units_lstm),  # Final LSTM layer—no sequences this time.
            Dropout(n_dropout),
            Dense(50, activation="relu"),  # Fully connected layer for some extra processing.
            Dense(20, activation="relu"),  # Another one.
            Dense(len(self.tickers))  # Output layer—one for each ticker.
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_lr), loss="mse")  # Compile with Adam optimizer.
        return model

    def tune_hyperparameters(self, X_train, y_train, X_val, y_val, max_trials=5):
        # Let’s find the best hyperparameters using Bayesian Optimization!
        tuner = kt.BayesianOptimization(
            lambda hp: self.build_model(hp, X_train.shape[1:]),
            objective='val_loss',  # Minimize validation loss.
            max_trials=max_trials,  # How many combos to try.
            executions_per_trial=1,
            directory='my_dir',
            project_name='lstm_tuning',
            seed=SEED
        )
        tuner.search(
            X_train, y_train,
            epochs=20,
            validation_data=(X_val, y_val),
            callbacks=[self.es]  # Early stopping to save time.
        )
        best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]  # Grab the winner.
        return best_hps

    def prepare_data_for_training(self, df_ret):
        # This ties everything together to get our data ready for training.
        X, y, dates = self.gen_predictors(df_ret)  # Generate predictors.
        X_train_scaled, X_val_scaled, X_test_scaled, y_train_scaled, y_val_scaled, y_test_scaled = self.prepare_data(X, y, dates)
        # Create lagged datasets for training, validation, and testing.
        self.X_train, self.y_train, self.train_time = self.create_lags(X_train_scaled, y_train_scaled, dates[dates <= pd.Timestamp("2017-12-31")][:len(y_train_scaled)])
        self.X_val, self.y_val, self.val_time = self.create_lags(X_val_scaled, y_val_scaled, dates[dates <= pd.Timestamp("2017-12-31")][len(y_train_scaled):])
        self.X_test, self.y_test, self.test_time = self.create_lags(X_test_scaled, y_test_scaled, dates[dates >= pd.Timestamp("2018-01-01")])

        # Make sure test_time matches the number of lagged samples.
        self.test_time = self.test_time[:len(self.y_test)]

        # Just checking our shapes to make sure everything lines up.
        print(f"X_test shape: {self.X_test.shape}")
        print(f"y_test shape: {self.y_test.shape}")
        print(f"test_time length: {len(self.test_time)}")
        print(f"y_test_original_lagged shape: {self.y_test_original_lagged.shape}")

    def train(self, tune=False, max_trials=50):
        # Time to train the model!
        if self.X_train is None or self.y_train is None:
            raise ValueError("Training data not prepared. Call prepare_data_for_training() first.")  # Safety check.

        if tune:
            # If we’re tuning, let’s find the best settings first.
            print("Starting hyperparameter tuning with Bayesian Optimization...")
            best_hps = self.tune_hyperparameters(self.X_train, self.y_train, self.X_val, self.y_val, max_trials)
            self.units_lstm = best_hps.get('units_lstm')
            self.n_dropout = best_hps.get('n_dropout')
            self.hp_lr = best_hps.get('hp_lr')
            print(f"Best hyperparameters found: units_lstm={self.units_lstm}, n_dropout={self.n_dropout}, hp_lr={self.hp_lr}")

        # Build the LSTM model with our chosen (or default) settings.
        self.model = Sequential([
            tf.keras.layers.Input(shape=(self.X_train.shape[1], self.X_train.shape[2])),
            LSTM(self.units_lstm, return_sequences=True),
            Dropout(self.n_dropout),
            LSTM(self.units_lstm, return_sequences=True),
            Dropout(self.n_dropout),
            LSTM(self.units_lstm),
            Dropout(self.n_dropout),
            Dense(50, activation="relu"),
            Dense(20, activation="relu"),
            Dense(len(self.tickers))
        ])
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.hp_lr), loss="mse")
        # Train it and keep track of the loss.
        history = self.model.fit(self.X_train, self.y_train, validation_data=(self.X_val, self.y_val),
                                epochs=self.n_epochs, callbacks=[self.es, self.lr_scheduler])

        self.model.save_weights('model_weights.weights.h5')  # Save the trained weights.

        # Plot how the loss changed over time—cool to see!
        plt.figure(figsize=(12, 6))
        sns.lineplot(x=range(len(history.history['loss'])), y=history.history['loss'], label='Train Loss')
        sns.lineplot(x=range(len(history.history['val_loss'])), y=history.history['val_loss'], label='Validation Loss')
        plt.legend()
        plt.title("Training and Validation Loss")
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.show()
        return history

    def predict(self, X):
        # Make predictions with our trained model.
        if self.model is None:
            if os.path.exists('model_weights.weights.h5'):
                # If the model isn’t loaded but weights exist, rebuild and load them.
                self.model = Sequential([
                    tf.keras.layers.Input(shape=(X.shape[1], X.shape[2])),
                    LSTM(self.units_lstm, return_sequences=True),
                    Dropout(self.n_dropout),
                    LSTM(self.units_lstm, return_sequences=True),
                    Dropout(self.n_dropout),
                    LSTM(self.units_lstm),
                    Dropout(self.n_dropout),
                    Dense(50, activation="relu"),
                    Dense(20, activation="relu"),
                    Dense(len(self.tickers))
                ])
                self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.hp_lr), loss="mse")
                self.model.load_weights('model_weights.weights.h5')
            else:
                raise ValueError("Model not trained. Call train() first.")  # No model, no predictions!
        pred_scaled = self.model.predict(X)  # Get scaled predictions.
        # Unscale them back to real-world values.
        pred = np.column_stack([self.scalers_output[ticker].inverse_transform(pred_scaled[:, i].reshape(-1, 1))
                              for i, ticker in enumerate(self.tickers)])
        return pred

    def analyze_performance(self):
        # Let’s see how well our model did!
        if any(var is None or (isinstance(var, np.ndarray) and var.size == 0) for var in [self.X_test, self.y_test, self.y_train]):
            raise ValueError("Test data not prepared. Ensure prepare_data_for_training() was called.")  # Safety first.

        pred_val = self.predict(self.X_val)  # Predictions for validation.
        pred_test = self.predict(self.X_test)  # Predictions for test.

        predictions_df = pd.DataFrame(pred_test, index=self.test_time, columns=self.tickers)  # Organize test predictions.
        actual_returns_df = pd.DataFrame(self.y_test, index=self.test_time, columns=self.tickers)  # Organize actual test returns.

        # Validation plot—comparing actual vs predicted returns over time.
        val_df = pd.DataFrame({
            'Date': np.tile(self.val_time, len(self.tickers)),
            'Actual': np.concatenate([self.scalers_output[ticker].inverse_transform(self.y_val[:, i].reshape(-1, 1)).flatten()
                                     for i, ticker in enumerate(self.tickers)]),
            'Predicted': np.concatenate([pred_val[:, i] for i in range(len(self.tickers))]),
            'Ticker': np.repeat(self.tickers, len(self.val_time))
        })
        g = sns.FacetGrid(val_df, col='Ticker', col_wrap=1, height=4, aspect=2, sharey=False)
        g.map(sns.lineplot, 'Date', 'Actual', label='Actual', color='blue', alpha=0.6)
        g.map(sns.lineplot, 'Date', 'Predicted', label='Predicted', color='orange', linestyle='--')
        g.add_legend()
        g.set_titles("{col_name}")
        g.fig.suptitle('Validation Predictions vs Actual Returns', y=1.02)
        for ax in g.axes.flat:
            ax.set_xlabel('Year')  # Now showing years!
            ax.set_ylabel('Return')
            ax.tick_params(axis='x', rotation=45)  # Rotate dates for readability.
        plt.subplots_adjust(hspace=0.5)
        plt.show()

        # Test plot—same deal, but for the test period.
        test_df = pd.DataFrame({
            'Date': np.tile(self.test_time, len(self.tickers)),
            'Actual': np.concatenate([self.scalers_output[ticker].inverse_transform(self.y_test[:, i].reshape(-1, 1)).flatten()
                                     for i, ticker in enumerate(self.tickers)]),
            'Predicted': np.concatenate([pred_test[:, i] for i in range(len(self.tickers))]),
            'Ticker': np.repeat(self.tickers, len(self.test_time))
        })
        g = sns.FacetGrid(test_df, col='Ticker', col_wrap=1, height=4, aspect=2, sharey=False)
        g.map(sns.lineplot, 'Date', 'Actual', label='Actual', color='blue', alpha=0.6)
        g.map(sns.lineplot, 'Date', 'Predicted', label='Predicted', color='orange', linestyle='--')
        g.add_legend()
        g.set_titles("{col_name}")
        g.fig.suptitle('Test Predictions vs Actual Returns', y=1.02)
        for ax in g.axes.flat:
            ax.set_xlabel('Year')  # Years here too!
            ax.set_ylabel('Return')
            ax.tick_params(axis='x', rotation=45)
        plt.subplots_adjust(hspace=0.5)
        plt.show()

        return predictions_df, actual_returns_df  # Return the dataframes for further analysis if needed.


In [ ]:
lstm_model = MultiOutputLSTM(tickers)
lstm_model.prepare_data_for_training(df_ret)
history = lstm_model.train(tune=True, max_trials=5)
print("Model training completed.")

predictions_df, actual_returns_df = lstm_model.analyze_performance()
print("Performance analysis completed.")

Performance analysis completed.
Best hyperparameters found:

units_lstm=300,
n_dropout=0.0,
hp_lr=0.007902373711581125
Loss: Training Loss: 0.0096 Val_loss: 0.0132

The neural network was able to find acceptible parameters for training and validation but it fail to generalize it for out of sample. The plots suggest that it was not capable of learning the patterns, it may be a case of overfitting or lack of complexity, also the use of low values for epochs and max trials in the tuner may have limited the results.

##3.c and d) Trading strategy for out-of-sample and Backtesting:

The TradingStrategy class is a Python implementation designed to backtest a predictive trading strategy against a passive buy-and-hold benchmark,

Backtest:

This method exemplifies a basic backtesting framework, integrating predictive signals with portfolio optimization.

Trading Strategy

The trading strategy mimics a long-short equity hedge fund approach, leveraging predictions to exploit relative performance. For simplicity some values were disconsidered (e.g., fixed weights, no transaction costs, no leverage constraints)

Every few days (based on a set frequency), the strategy looks at predictions for how assets will perform. It picks the top 2 assets expected to do well and invests half the portfolio in each (these are "long" positions). Then, it picks the bottom 2 assets expected to do poorly and bets against them by assigning each a negative weight (these are "short" positions). All other assets stay at zero.

Buy-and-Hold Strategy

Setup: Takes the same starting money and splits it equally across all assets—like buying a little bit of everything and holding onto it.

Growth Over Time: Watches how each asset performs day by day, letting the initial investment grow or shrink naturally based on those daily changes.

Portfolio Value: Adds up the value of all assets each day to see how the total portfolio is doing.

In [ ]:
class TradingStrategy:
    def __init__(self, predictions_df, daily_returns_df, assets, initial_investment=10000, rebalance_freq=5):
      # Initialize instance variables with input data and parameters
        self.predictions_df = predictions_df
        self.daily_returns = daily_returns_df
        self.assets = assets
        self.initial_investment = initial_investment
        self.rebalance_freq = rebalance_freq

        # Convert returns to decimal if in percentage form
        if self.daily_returns.max().max() > 1:
            self.daily_returns = self.daily_returns / 100

    def backtest(self):
      # Align dates between predictions and returns DataFrames
        dates = self.predictions_df.index.intersection(self.daily_returns.index)
        preds = self.predictions_df.loc[dates]
        rets = self.daily_returns.loc[dates].fillna(0)  # Handle NaNs


        # Trading strategy: long top 2 and short bottom 2 assets based on predictions
        strategy_value = self.initial_investment
        weights = {asset: 0 for asset in self.assets}
        strategy_values = []

        # Iterate over dates to get strategy returns
        for i, date in enumerate(dates):
            if i % self.rebalance_freq == 0:
                predictions = preds.loc[date]
                ranked = predictions.sort_values(ascending=False).index
                long_assets = ranked[:2]
                short_assets = ranked[-2:]
                weights = {asset: 0.5 if asset in long_assets else -0.5 if asset in short_assets else 0
                          for asset in self.assets}
            # Calculate daily strategy return and update portfolio value
            daily_ret_strategy = sum(weights[asset] * rets.loc[date, asset] for asset in self.assets)
            strategy_value *= (1 + daily_ret_strategy)
            strategy_value = max(strategy_value, 0)
            strategy_values.append(strategy_value)


        # Buy-and-hold strategy: equal investment in all assets
        initial_per_asset = self.initial_investment / len(self.assets) # Initial investment per asset
        cum_returns = (1 + rets / 100).cumprod(axis=0) - 1  # Cumulative growth factor (returns are in decimal form)
        asset_values = (1 + cum_returns) * initial_per_asset  # Value of each asset over time
        bh_daily_values = asset_values.sum(axis=1)  # Total portfolio value

        # Create results DataFrame
        results = pd.DataFrame({
            'Strategy': strategy_values,
            'Buy-and-Hold': bh_daily_values
        }, index=dates)

        # Calculate and print cumulative returns
        strat_cum_return = (results['Strategy'].iloc[-1] / self.initial_investment - 1) * 100
        bh_cum_return = (results['Buy-and-Hold'].iloc[-1] / self.initial_investment - 1) * 100
        print(f"Cumulative Return - Trading Strategy: {strat_cum_return:.2f}%")
        print(f"Cumulative Return - Buy-and-Hold: {bh_cum_return:.2f}%")

        return results

    def plot(self):
        """Plot the backtest results."""

        results = self.backtest()
        plot_df = results.reset_index().melt(id_vars=['index'], var_name='Strategy', value_name='Value')
        plt.figure(figsize=(10, 5))
        sns.lineplot(data=plot_df, x='index', y='Value', hue='Strategy')
        plt.title('Trading Strategy vs Buy-and-Hold')
        plt.xlabel('Date')
        plt.ylabel('Portfolio Value')
        plt.show()


In [ ]:
strategy = TradingStrategy(predictions_df, actual_returns_df, tickers)
strategy.plot()

The Strategy did not perform well. This may be for the fact that all the assets show some postive returns over time as it can be seen in the buy and hold strategy so shorting assets with positive returns (even limited ones) may have result in the negative trend.

Another point is that the neural network did not perform well in testing data so it is unlikely that the predictions reflect the best possible strategies.

The long-short strategy from step 2 performed better.